In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision.models import resnet50
from torch.utils.data import DataLoader, Subset
from torch.optim import Adam
from torch.nn import CrossEntropyLoss
import numpy as np
from sklearn.metrics import classification_report


model = resnet50(pretrained=True)


for param in model.parameters():
    param.requires_grad = False


num_classes = 10
model.fc = torch.nn.Linear(model.fc.in_features, num_classes)


transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


full_train_set = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_set = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)


np.random.seed(42)
train_indices = np.random.choice(len(full_train_set), 2000, replace=False)
test_indices = np.random.choice(len(test_set), 1000, replace=False)

train_set = Subset(full_train_set, train_indices)
test_set = Subset(test_set, test_indices)


train_loader = DataLoader(train_set, batch_size=64, shuffle=True)
test_loader = DataLoader(test_set, batch_size=64, shuffle=False)

# Define the loss function and optimizer
criterion = CrossEntropyLoss()
optimizer = Adam(model.fc.parameters(), lr=0.001)

# Function to evaluate the model
def evaluate_model(model, loader):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for images, labels in loader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            all_preds.extend(predicted.numpy())
            all_labels.extend(labels.numpy())

    # Print classification report
    print(classification_report(all_labels, all_preds, target_names=[f'Class {i}' for i in range(num_classes)]))

# Training loop
def train_model(model, criterion, optimizer, num_epochs=10):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}')

        evaluate_model(model, test_loader)

# Start training
train_model(model, criterion, optimizer, num_epochs=5)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:01<00:00, 63.5MB/s]


100%|██████████| 170498071/170498071 [00:11<00:00, 14880368.02it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
Epoch 1/5, Loss: 1.7740797773003578
              precision    recall  f1-score   support

     Class 0       0.56      0.80      0.66        90
     Class 1       0.62      0.95      0.75       115
     Class 2       0.49      0.76      0.60       118
     Class 3       0.77      0.53      0.63       103
     Class 4       0.70      0.60      0.65       106
     Class 5       0.70      0.72      0.71       102
     Class 6       0.68      0.80      0.73        93
     Class 7       0.88      0.57      0.69        86
     Class 8       0.87      0.30      0.44        87
     Class 9       1.00      0.50      0.67       100

    accuracy                           0.66      1000
   macro avg       0.73      0.65      0.65      1000
weighted avg       0.72      0.66      0.65      1000

Epoch 2/5, Loss: 1.112622432410717
              precision    recall  f1-score   support

     Class 0       0.79   